# Python 最优化算法

* 需要提前配置 Gurobi 库
* 因为需要更换环境，所以本报告单列

Task-4 内容包括
1. 入门级实例 : 线性规划问题实例(及非线性约束)
2. 更多约束 : TSP问题求解

In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

## 随意设置的线性规划问题实例

In [2]:
# 加载数据
data = pd.read_csv("./2022期末数据/11 (1).tsv" , sep = "\t")
data.head()

,dataset,tpm,tmm
0,gtex_Adipose_Subcutaneous,0.133341,0.028823
1,gtex_Adipose_Visceral_Omentum,0.230250,0.040660
2,gtex_Adrenal_Gland,0.233416,0.054721
3,gtex_Artery_Aorta,0.080029,0.040266
4,gtex_Artery_Coronary,0.130536,0.036145


In [3]:
# 配置模型
model = gurobipy.Model("LP Model")

# 定义自变量
tpm = model.addVar(lb=0.0, ub = gurobipy.GRB.INFINITY, vtype=gurobipy.GRB.CONTINUOUS, name = "tpm")
tmm = model.addVar(lb=0.0, ub = gurobipy.GRB.INFINITY, vtype=gurobipy.GRB.CONTINUOUS, name = "tmm")

# 设置目标函数
model.setObjective(20 * tpm + 10 * tmm, gurobipy.GRB.MAXIMIZE) # 最大化
# 设置11个约束
for i in range(5):
    model.addConstr(float(data.iloc[i][['tpm']].values) * tpm + float(data.iloc[i][['tmm']].values) * tmm <= 10)
    model.addConstr(float(data.iloc[i+5][['tpm']].values) * tpm + float(data.iloc[i+5][['tmm']].values) * tmm >= 1)
model.addConstr(tmm <= 80)
model.update()

# 显示求解过程（套路代码）
model.Params.LogToConsole = True
# 开始优化求解
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 11 rows, 2 columns and 21 nonzeros
Model fingerprint: 0x0ce9ea1c
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+01]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 2 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4594011e+03   2.420062e+01   0.000000e+00      0s
       2    1.2817420e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.281742021e+03


In [4]:
print(f"optimal objective value is {model.objVal}.")
# 查看变量取值
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))

optimal objective value is 1281.7420207561472.
tpm 24.0871
tmm 80


非线性约束实例

In [5]:
# 配置模型
model = gurobipy.Model("Model 2")

# 定义自变量
tpm = model.addVar(lb=0.0, ub = gurobipy.GRB.INFINITY, vtype=gurobipy.GRB.CONTINUOUS, name = "tpm")
tmm = model.addVar(lb=0.0, ub = gurobipy.GRB.INFINITY, vtype=gurobipy.GRB.CONTINUOUS, name = "tmm")

# 设置目标函数
model.setObjective(20 * tpm + 10 * tmm, gurobipy.GRB.MAXIMIZE) # 最大化
# 设置非线性约束
for i in range(5):
    model.addConstr(float(data.iloc[i][['tpm']].values) * tpm**2 + float(data.iloc[i][['tmm']].values) * tmm**2<= 100)
    # f非线性约束中必须是整数幂 , float 类型会报错
    model.addConstr(float(data.iloc[i+5][['tpm']].values) * tpm + float(data.iloc[i+5][['tmm']].values) * tmm >= 1)
model.update()

# 显示求解过程（套路代码）
model.Params.LogToConsole = True
# 开始优化求解
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 2 columns and 10 nonzeros
Model fingerprint: 0xe06fe8d1
Model has 5 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-02, 6e-01]
  QMatrix range    [3e-02, 2e-01]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [1e+02, 1e+02]
Presolve time: 0.00s
Presolved: 20 rows, 15 columns, 33 nonzeros
Presolved model has 5 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Dense cols : 1
 AA' NZ     : 1.080e+02
 Factor NZ  : 2.120e+02
 Factor Ops : 2.874e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.51761168e+01  4.51761168e+01  3.65e+00 1.78e+01  9.98e+01     0s
   1   6.124

In [6]:
print(f"optimal objective value is {model.objVal}.")
# 查看变量取值
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))

optimal objective value is 595.0737937549338.
tpm 14.3987
tmm 30.71


## 化为 TSP 问题

In [7]:
# 设置要读取的城市数量
city_num = 30

coordinate = data.iloc[:city_num][['tpm','tmm']]
coordinate = np.array(coordinate)

# 计算城市之间的距离
distance = np.zeros((city_num, city_num))
for i in range(city_num):
    for j in range(city_num):
        if (i == j):
            distance[i,j] = 10000
        else:
            distance[i,j] = np.sqrt(np.square(coordinate[i,0] - coordinate[j,0]) + np.square(coordinate[i,1]-coordinate[j,1]))

In [8]:
model = gurobipy.Model('TSP')

x = model.addVars(city_num, city_num, vtype = gurobipy.GRB.BINARY, name = 'x')
utility = model.addVars(city_num, lb = 0, vtype = gurobipy.GRB.CONTINUOUS, name = 'utility')

model.setObjective(sum(x[i,j] * distance[i,j] for i in range(city_num) for j in range(city_num)), GRB.MINIMIZE)

# 设置约束
for j in range(city_num):    
    model.addConstr(gurobipy.quicksum(x[i,j] for i in range(city_num)) == 1)
for i in range(city_num):
    for j in range(1, city_num):
        if (i != j):
            model.addConstr(utility[i] - utility[j] + city_num * x[i,j] <= city_num - 1)
model.update()

# 显示求解过程
model.Params.LogToConsole = True
# 限制求解时间
model.Params.TimeLimit =50
# 开始优化求解
model.optimize()

Set parameter TimeLimit to value 50
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 871 rows, 930 columns and 3423 nonzeros
Model fingerprint: 0x7982bd32
Variable types: 30 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [7e-03, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 300000.00000
Presolve removed 1 rows and 31 columns
Presolve time: 0.00s
Presolved: 870 rows, 899 columns, 3364 nonzeros
Found heuristic solution: objective 290000.00784
Variable types: 29 continuous, 870 integer (870 binary)

Root relaxation: objective 9.334064e-01, 62 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 

In [9]:
print(f"optimal objective value is {model.objVal}.")

# 将结果输出成路径
u_value = []
for i in range(city_num):
    u_value.append(utility[i].x) # 加载 utility[i] 的值
sequence = sorted(enumerate(u_value), key = lambda y:y[1]) # 按照 utility 值排序
for item in sequence:
    print(item[0] + 1,'->',end=' ')
print(1)

optimal objective value is 1.136092581758076.
1 -> 5 -> 26 -> 7 -> 30 -> 25 -> 28 -> 29 -> 2 -> 22 -> 3 -> 4 -> 6 -> 21 -> 23 -> 27 -> 12 -> 24 -> 11 -> 13 -> 14 -> 8 -> 9 -> 10 -> 20 -> 16 -> 17 -> 15 -> 18 -> 19 -> 1
